In [6]:

print(np.load("scores.model_idx_0.npz")["iptm"])


[0.29265615]


In [35]:
# Load the .npz file
data = np.load('/home/ubuntu/chai-lab/examples/restraints/5hps/5hps_restraints_1.csv_Chai1_output/scores.model_idx_0.npz')

# Print all keys in the .npz file
print(data.files)

# Optionally, iterate through keys and access the corresponding arrays
for key in data.files:
    print(f"Array with key '{key}':")
    print(data[key])
print(data['iptm'])

['aggregate_score', 'ptm', 'iptm', 'per_chain_ptm', 'per_chain_pair_iptm', 'has_inter_chain_clashes', 'chain_chain_clashes', 'pae', 'pde', 'plddt']
Array with key 'aggregate_score':
[0.7073075]
Array with key 'ptm':
[0.784968]
Array with key 'iptm':
[0.6878923]
Array with key 'per_chain_ptm':
[[0.78667915 0.8481177 ]]
Array with key 'per_chain_pair_iptm':
[[[0.78667915 0.6878923 ]
  [0.584059   0.8481177 ]]]
Array with key 'has_inter_chain_clashes':
[False]
Array with key 'chain_chain_clashes':
[[[0 0]
  [0 0]]]
Array with key 'pae':
[[ 0.28342894  2.5119298   4.166955   ... 28.328255   28.42338
  28.57592   ]
 [ 0.99033093  0.25654998  1.4447242  ... 27.710173   27.841427
  28.293076  ]
 [ 2.8982646   1.2712972   0.25673306 ... 27.298664   27.423758
  27.595165  ]
 ...
 [27.410345   26.971745   26.81003    ...  0.3230196   1.5810688
   3.4697433 ]
 [27.375923   27.08361    26.986994   ...  1.4729072   0.26411012
   1.8093574 ]
 [27.347883   27.116795   27.106419   ...  3.606999    1.7

In [18]:
import numpy as np
import os
from matplotlib import pyplot as plt

#parse Chai-1 output cif file and extract the coordinates
candidate_cif = parse_CIF(str(chai_cif_file), ca_only = True)

# List the keys of the dictionary
print(candidate_cif.keys())

#extract chain A, B(heavy), and C(light) coordinates
A_candidate_coords = np.array(candidate_cif['coords_chain_A']['CA_chain_A'])
B_candidate_coords = np.array(candidate_cif['coords_chain_B']['CA_chain_B'])

print(A_candidate_coords.shape)
print(B_candidate_coords.shape)

#calculate distance between chain A and the concatenated chain B
distance_A_B = distance(A_candidate_coords, B_candidate_coords)

A_B_contact = distance_A_B < 8
A_B_contact_ind = np.array(np.where(A_B_contact))

print(len(A_B_contact_ind[0]))

# Load the .npz file
protein = '5hps'
patchdock_number = 1
base = '~/chai-lab/examples/restraints'

# Print all keys in the .npz file
#print(data.files)

# Optionally, iterate through keys and access the corresponding arrays
for key in data.files:
    print(f"Array with key '{key}':")
    print(data[key])

#Calculate DockQ score comparison of the Chai-1 prediction v. PDB groundtruth
    predicted_model_path = os.path.expanduser(f"~/chai-lab/examples/restraints/{protein}/{protein}_restraints_{patchdock_number}.csv_Chai1_output/pred.model_idx_0.cif")
    ground_truth_model_path = os.path.expanduser(f"~/chai-lab/examples/restraints/{protein}/{protein}.cif")
    
    # Run DockQ and get the results
    try:
        dockq_results = run_dockq(predicted_model_path, ground_truth_model_path)
        print(dockq_results)
    except Exception as e:
        print(f"An error occurred: {e}")

    npz_file = os.path.expanduser(f"{base}/{protein}/{protein}_restraints_{patchdock_number}.csv_Chai1_output/scores.model_idx_0.npz")
    pae = np.load(npz_file)["pae"]
    print(pae, pae.shape)
    ipae = pae[A_B_contact_ind_shifted[0], A_B_contact_ind_shifted[1]]
    mean_ipae = np.mean(ipae)
    print(mean_ipae)


#reformat iPAE
plt.imshow(pae, cmap="bwr", vmin=0, vmax=32) #establish min and max for PAE
plt.colorbar(label='Expected position error (Angstroms)')
plt.title(f"{protein} - Patch {patchdock_number}")
plt.xlabel('Scored Residue')
plt.ylabel('Aligned Residue')
plt.text(0.5, -0.15, f"DockQ Score: {dockq_results['A']['B']['DockQ']}", ha='center', va='top', transform=plt.gca().transAxes)
plt.show()

print('DockQ Score:', dockq_results['A']['B']['DockQ'])

NameError: name 'chai_cif_file' is not defined